In [1]:
!pip install sentence-transformers pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import pinecone
from pinecone import Pinecone, ServerlessSpec

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# 1. 모델 로드 (Sentence-BERT)
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
import numpy as np

# multi-qa-mpnet-base-dot-v1 모델 로드
#model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
model.get_sentence_embedding_dimension()

384

In [5]:
# 2. 파인콘 초기화
from pinecone import Pinecone, ServerlessSpec

pinecone = Pinecone(api_key="")
index = pinecone.Index("trip-index")

In [6]:
# 3. 검색 함수 정의

# 여행 스타일 및 동행인 기반
def search_places_style(city, companions, travel_style):
    query = f"Best places in {city} for {companions} with focus on {travel_style}."
    query_embedding = model.encode(query).tolist()
    namespace = f"{city}_tour"
    results_style= index.query(vector=query_embedding, top_k=30, namespace=namespace, include_metadata=True)
    return results_style

# 기본 유명 관광지
def search_places(city):
    query = f"The most famous tour places in {city}"
    query_embedding = model.encode(query).tolist()
    namespace = f"{city}_tour"
    results_best= index.query(vector=query_embedding, top_k=50, namespace=namespace, include_metadata=True)
    return results_best

In [7]:
# 쿼리 벡터 생성 - 파리 / 여행일정
city = 'paris'
companions = 'parents'
travel_style = 'shopping', 'dining', 'nightlife'
results_style = search_places_style(city, companions, travel_style)
results_best = search_places(city)

In [8]:
print(len(results_style['matches']))
print(len(results_best['matches']))

30
50


In [9]:
import pandas as pd

# 검색 결과를 CSV 파일로 저장하는 함수
def save_results_to_csv(results_style, results_best, file_name="search_results.csv"):
    # 결과에서 필요한 데이터를 추출

    # results_style
    data1 = []
    for result in results_style['matches']:
        row = {
            'ID': result['id'],
            'Score': result['score']
        }
        # 메타데이터가 포함되어 있을 경우 추가
        row.update(result.get('metadata', {}))
        data1.append(row)

    # results_best
    data2 = []
    for result in results_best['matches']:
        row = {
            'ID': result['id'],
            'Score': result['score']
        }
        # 메타데이터가 포함되어 있을 경우 추가
        row.update(result.get('metadata', {}))
        data2.append(row)

    # pandas DataFrame으로 변환
    df1 = pd.DataFrame(data1)
    df2 = pd.DataFrame(data2)
    df = pd.concat([df1, df2], ignore_index=True)
    df = df.drop_duplicates(subset=['ID'])     # 중복해서 나온 장소 제거

    # CSV 파일로 저장
    df.to_csv(file_name, index=False)
    print(f"Results saved to {file_name}")

# 검색 결과를 CSV 파일로 저장
save_results_to_csv(results_style, results_best, file_name="paris_search_results.csv")

Results saved to paris_search_results.csv


In [10]:
results_style

{'matches': [{'id': 'places_352',
              'metadata': {'1_이름': 'Start Marais shopping',
                           '2_주소': 'Rue Vieille du Temple, 75003 Paris, France',
                           '3_평점': '4.8',
                           '4_위도': 48.8599931,
                           '5_경도': 2.3610306,
                           '6_리뷰': 'Go my self this hat and other thing . The '
                                   'price were affordable even do is a very '
                                   'busy part of Paris . A lot of awesome '
                                   'places to eat and drink . Paris very good '
                                   'service amazing place ',
                           '7_영업시간': 'Monday: 9:30\u202fAM\u2009–\u2009'
                                     '7:00\u202fPM\n'
                                     'Tuesday: 9:30\u202fAM\u2009–\u2009'
                                     '7:00\u202fPM\n'
                                     'Wednesday: 9:30\u202fA

In [3]:
# install
!pip install --upgrade openai
!pip install langchain
!pip install langchain_openai

!pip install langchain[all]

from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate
import getpass
import os
from langchain_openai import ChatOpenAI
import pandas as pd
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

import pandas as pd
import re
import requests
import openai

!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.

In [4]:
import os
os.environ["OPENAI_API_KEY"] =""

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4o",  # 모델명
)


import openai

client = openai.OpenAI()

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "나의 프롬프트 엔지니어링 코치가 되어줘."},
        {"role": "user", "content": "어떻게 하면 프롬프트 엔지니어링을 잘 할수 있을까?"}
    ]
)

print("Assistant: " + response.choices[0].message.content)

Assistant: 프롬프트 엔지니어링을 잘하기 위해서는 다음의 몇 가지 핵심 원칙과 전략을 이해하고 실천하는 것이 중요합니다.

1. **명확성과 구체성**:
   - 가능한 한 명확하고 구체적으로 프롬프트를 작성하세요. 원하는 정보를 정확하게 정의하면 원하는 결과를 얻기 쉽습니다.

2. **맥락 제공**:
   - AI에게 필요한 배경 정보를 제공합니다. 맥락이 충분하면 더 관련성 있는 응답을 받을 수 있습니다.

3. **목적 의식**:
   - 프롬프트를 작성할 때 목표를 명확히 하세요. 무슨 결과를 얻고자 하는지를 명확하게 알고 있어야 합니다.

4. **구조화된 방식 활용**:
   - 복잡한 정보를 요청할 경우, 리스트나 단계별로 작성하여 AI가 쉽게 따를 수 있도록 합니다.

5. **테스트 및 반복**:
   - 다양한 프롬프트를 테스트하고 결과를 비교해 보세요. 가장 효과적인 구문이나 형식을 찾기 위해 여러 번 시도할 필요가 있습니다.

6. **구체적인 예시 제공**:
   - 특히 복잡한 질문을 할 때는 예시를 제공하여 AI가 이해할 수 있도록 합니다.

7. **피드백 활용**:
   - AI가 제공한 응답을 검토하고, 원하는 방향으로 더욱 개선된 프롬프트를 작성하도록 해 보세요.

8. **제한사항 명시**:
   - 특정 정보만을 원할 때는 제한사항을 명시하여 불필요한 응답을 피할 수 있습니다.

9. **실험과 학습**:
   - 다양한 도메인과 주제에서 실험해 보면서, AI의 반응을 학습하고 프롬프트를 조정합니다.

이러한 전략들을 실천하면 프롬프트 엔지니어링에서 점점 더 뛰어난 결과를 얻을 수 있을 것입니다.


In [ ]:
# 2. CSV 파일에서 데이터 로드
def load_places_from_csv(file_path):
    return pd.read_csv(file_path)

In [48]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
# 페르소나 설정
persona = """
You are a travel AI expert specialized in generating optimized travel itineraries.

You always generate travel itineraries based on verifiable factual statements.
You mainly speak about factual information related to tourist attractions and restaurants, and recent updates, without adding information on your own.
"""

prompt_template = """
당신은 여행 일정 AI 전문가입니다. {city}에서 {trip_duration} 동안의 여행 일정을 생성하세요.
여행 정보는 아래와 같습니다:

- 여행 날짜: {travel_dates}
- 동행인: {companions}
- 여행 스타일: {travel_style}
- 선호 일정 스타일: {itinerary_style}

**아래 장소 리스트에서 조건에 맞게 장소를 조합해 여행 일정을 만들어주세요**:
{places_list}

**제약 조건**:
1. 오전, 오후, 저녁으로 구분된 일정을 만드세요.
2. 동일한 장소나 식당은 **여행 일정에 중복되지 않도록** 하세요. 동일한 장소는 반복되지 않아야 합니다. 다른 날짜에 가기로한 장소는 다시 나올 수 없습니다.
3. 영업 시간과 주소를 일정에 포함하세요.
4. 각 장소에 대해 한 문장으로 간략한 소개를 추가하세요.
5. 동선 최적화를 고려해 이동 시간을 최소화하세요.
6. 각 장소의 **운영 시간**을 고려하여 일정을 구성하세요.
7. 여행 스타일에 따라 일정의 밀도를 조정하세요.
  7-1. 선호하는 일정 스타일로 ‘빼곡한 일정’을 선택한 경우 오전, 오후, 저녁에 따라 관광지 2개, 맛집 1개를 포함해서 하루에 총 9개의 활동을 포함하세요.
  7-2. 선호하는 일정 스타일로 ‘널널한 일정’을 선택한 경우 오전, 오후, 저녁에 따라 관광지 1개, 맛집 1개를 포함해서 하루에 총 6개의 활동을 포함하세요.
8. 결과는 아래 출력 구조를 참고하여 JSON 형식으로 출력하세요.
9. **결과는 한국어로 제공해주세요.**

**출력 구조:**
- 날짜, 시간대, 장소명, 장소 소개, 운영시간, 주소가 출력될 수 있도록 해주세요.
"""

# 여행 일정 생성 함수
def generate_travel_itinerary(df, places_list,city, trip_duration, travel_dates, companions, travel_style, itinerary_style):
    # 페르소나 주입
    filled_persona = persona.format()

    # 장소 리스트를 문자열로 변환
    places_list = "\n".join([
        f"- {row['1_이름']} (카테고리: {row['8_유형']}, 주소: {row['2_주소']}, 운영시간: {row.get('7_영업시간', 'N/A')})"
        for _, row in df.iterrows()
    ])

    formatted_prompt = prompt_template.format(
        city=itinerary_details['city'],
        trip_duration=itinerary_details['trip_duration'],
        travel_dates=itinerary_details['travel_dates'],
        companions=itinerary_details['companions'],
        travel_style=itinerary_details['travel_style'],
        itinerary_style=itinerary_details['itinerary_style'],
        places_list = places_list
    )


    # 프롬프트 구성
    prompt = ChatPromptTemplate(
        template=formatted_prompt,
        messages=[
            SystemMessagePromptTemplate.from_template(filled_persona),  # 페르소나 주입
            MessagesPlaceholder(variable_name="chat_history"),  # 메모리 활용
            HumanMessagePromptTemplate.from_template("{question}")
        ]
    )

    # 메모리 설정
    memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=1000, memory_key="chat_history", return_messages=True)

    # LLMChain 설정
    conversation = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=True,
        memory=memory
    )

    # 여러 입력 변수를 하나의 딕셔너리로 전달
    result = conversation({
        "question": formatted_prompt
    })

    return result['text']


# 딕셔너리 형태로 일정 정보 전달
itinerary_details = {
    "city": "프랑스 파리",
    "trip_duration": "3박 4일",
    "travel_dates": "2024-11-15 ~ 2024-11-18",
    "companions": "친구와",
    "travel_style": "쇼핑, 관광지, 맛집",
    "itinerary_style": "빼곡한 일정"
}

# CSV 파일에서 장소 데이터 로드
df = load_places_from_csv("paris_search_results.csv")
places_list = "\n".join([
        f"- {row['1_이름']} (카테고리: {row['8_유형']}, 주소: {row['2_주소']}, 운영시간: {row.get('7_영업시간', 'N/A')})"
        for _, row in df.iterrows()
    ])
# 여행 일정 생성 호출
itinerary = generate_travel_itinerary(df, places_list,**itinerary_details)

# 결과 출력
print(itinerary)





> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a travel AI expert specialized in generating optimized travel itineraries.

You always generate travel itineraries based on verifiable factual statements.
You mainly speak about factual information related to tourist attractions and restaurants, and recent updates, without adding information on your own.

Human: 
당신은 여행 일정 AI 전문가입니다. 프랑스 파리에서 3박 4일 동안의 여행 일정을 생성하세요. 
여행 정보는 아래와 같습니다:

- 여행 날짜: 2024-11-15 ~ 2024-11-18
- 동행인: 친구와
- 여행 스타일: 쇼핑, 관광지, 맛집
- 선호 일정 스타일: 빼곡한 일정

**아래 장소 리스트에서 조건에 맞게 장소를 조합해 여행 일정을 만들어주세요**:
- Start Marais shopping (카테고리: shopping_mall, point_of_interest, establishment, 주소: Rue Vieille du Temple, 75003 Paris, France, 운영시간: Monday: 9:30 AM – 7:00 PM
Tuesday: 9:30 AM – 7:00 PM
Wednesday: 9:30 AM – 7:00 PM
Thursday: 9:30 AM – 7:00 PM
Friday: 9:30 AM – 7:00 PM
Saturday: 9:30 AM – 7:00 PM
Sunday: Closed)
- Galerie du Claridge (카테고리: shopping_mall, point_of_interest, establishment, 주소: 74 Av.

In [56]:
import json
# 불필요한 설명 문장 및 뒤쪽 텍스트 제거하고 JSON 부분만 추출
start_index = itinerary.find("{")  # JSON 시작 위치 찾기
end_index = itinerary.rfind("}")   # JSON 끝 위치 찾기

# JSON 부분만 추출
json_text = itinerary[start_index:end_index+1]

# JSON 문자열을 파싱하여 Python 객체로 변환
data = json.loads(json_text)

# "여행 일정" 키 아래의 리스트를 DataFrame으로 변환
df = pd.DataFrame(data["여행 일정"])
df

,날짜,시간대,장소명,장소 소개,운영시간,주소
0,2024-11-15,오전,Galerie Vivienne,19세기 파리의 매력을 간직한 아름다운 쇼핑 아케이드입니다.,8:30 AM – 8:00 PM,"4 Rue des Petits Champs, 75002 Paris, France"
1,2024-11-15,오전,Domaine National du Palais-Royal,역사적인 궁전과 정원이 있는 관광 명소입니다.,8:30 AM – 10:30 PM,"75001 Paris, France"
2,2024-11-15,오전,Boulangerie LIBERTÉ,신선한 빵과 페이스트리를 제공하는 인기 있는 빵집입니다.,7:30 AM – 8:00 PM,"58 Rue Saint-Dominique, 75007 Paris, France"
3,2024-11-15,오후,Petit Palais,아름다운 건축물과 예술 작품을 감상할 수 있는 미술관입니다.,10:00 AM – 6:00 PM,"Av. Winston Churchill, 75008 Paris, France"
4,2024-11-15,오후,Galeries Lafayette Champs-Élysées,고급 브랜드와 최신 패션을 만날 수 있는 쇼핑 명소입니다.,10:00 AM – 9:00 PM,"60 Av. des Champs-Élysées, 75008 Paris, France"
5,2024-11-15,오후,Le Café Marly,루브르 박물관을 바라보며 식사를 즐길 수 있는 카페입니다.,8:00 AM – 2:00 AM,"93 Rue de Rivoli, 75001 Paris, France"
6,2024-11-15,저녁,Place de la Bastille,"프랑스 혁명의 상징적인 장소로, 역사적인 의미가 깊습니다.",운영 시간 없음,"Pl. de la Bastille, 75004 Paris, France"
7,2024-11-15,저녁,Le Bon Marché,"파리에서 가장 오래된 백화점 중 하나로, 다양한 상품을 제공합니다.",10:00 AM – 7:45 PM,"24 Rue de Sèvres, 75007 Paris, France"
8,2024-11-15,저녁,Brasserie Bellanger,전통적인 프랑스 요리를 맛볼 수 있는 브라세리입니다.,9:00 AM – 12:00 AM,"140 Rue du Faubourg Poissonnière, 75010 Paris,..."
9,2024-11-16,오전,Start Marais shopping,트렌디한 패션과 독특한 상점들이 모여 있는 쇼핑 거리입니다.,9:30 AM – 7:00 PM,"Rue Vieille du Temple, 75003 Paris, France"


In [57]:
df['장소명'].nunique()

20